# Label using activity data

Labeling the modeled backbones with the activity predictor.   
The labeling can be used to select best fragmetns (see ```select_bb_fragments.ipynb```) instead of using Rosetta scores as labels 

In [1]:
import sys
import pandas as pd
import glob
import seaborn as sns
import numpy as np
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from math import floor
import os
import re
from itertools import product
from flab.rosetta.rosetta_output.resfile import ResFile
from matplotlib import rc
import matplotlib.patches as mpatches
sns.set()

# Parsing

In [2]:
blades_names = ['blade1', 'blade2_4', 'blade5_6', 'blade7_8']

Loading all scores of the chimeras

In [3]:
scores = pd.read_csv('../data/scores_all_chimeras_rnd2.csv.gz')
scores.set_index('description', inplace=True)
scores['description'] = scores.index
scores['blade1'] = scores.description.apply(lambda x: x.split('__')[0])
scores['blade2_4'] = scores.description.apply(lambda x: x.split('__')[1])
scores['blade5_6'] = scores.description.apply(lambda x: x.split('__')[2])
scores['blade7_8'] = scores.description.apply(lambda x: x.split('__')[3])
scores['normed_total_score'] = scores['total_score'] / scores['nres']

In [4]:
wt_label = '3w24_template__3w24_template__3w24_template__3w24_template'
WT = scores.loc[wt_label]
WT

total_score                                                          -1050.83
contacts                                                                 1537
dslf_fa13                                                                   0
fa_atr                                                               -2077.07
fa_dun                                                                401.249
fa_elec                                                              -583.758
fa_intra_rep                                                            3.938
fa_intra_sol_xover4                                                    69.006
fa_rep                                                                267.557
fa_sol                                                                1188.41
hbond_bb_sc                                                           -56.959
hbond_lr_bb                                                           -58.978
hbond_sc                                                        

# Assigning labels using ml model trained on repertoire 1 data

## Step 1: features for the logit model

In [5]:
logit_features = scores[['total_score', 'contacts', 'yhh_planarity', 
                         'p_aa_pp', 
                         'hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb',
                         'nres']].copy()

In [6]:
normed_cols = ['total_score', 'contacts', 'yhh_planarity',
               'hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb',
               'p_aa_pp']

for c in normed_cols:
    logit_features[c] = logit_features[c] / logit_features['nres']  

logit_features['hbond'] = logit_features[['hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb']].sum(axis=1)    
logit_features = logit_features[['total_score', 'hbond', 'p_aa_pp', 'yhh_planarity', 'contacts']]

logit_features.sample(10)

,total_score,hbond,p_aa_pp,yhh_planarity,contacts
description,,,,,
3wubA__3wufA__1ta3B__2depB,-2.546663,-0.888837,-0.193564,0.001031,4.076687
1nq6A__1ur2A__4f8xA__1uqzA,-2.639948,-0.864557,-0.216141,0.000445,4.433908
1us3A__4hu8E__3ro8D__1e5nB,-2.582309,-0.899680,-0.192710,0.000345,4.125348
3w24_template__4k68B__4w8lB__3emcA,-2.919425,-0.957518,-0.219870,0.000593,4.653614
5ay7A__4w8lC__4pmdA__4pmyA,-2.286078,-0.828508,-0.217941,0.000670,4.149533
4w8lC__3wufA__4xx6B__4qdmA,-2.697079,-0.868515,-0.207642,0.001258,4.700000
3muiA__3ro8B__1b3zA__1v6yA,-2.426925,-0.816207,-0.202464,0.001021,3.934132
1us3A__2wysB__3u7bC__1nq6A,-2.481695,-0.870413,-0.228348,0.001342,3.840456
1b3xA__1ta3B__3u7bC__3w24_template,-2.578856,-0.892125,-0.211169,0.001772,4.150000


The cell below is to be used if using the activity model including activeaite features

In [ ]:
# logit_features = scores[['total_score', 'contacts', 'yhh_planarity',
#                          'fa_rep_catres', 'degree_activesite', 
#                          'p_aa_pp', 'p_aa_pp_activesite', 'p_aa_pp_catres', 
#                          'fa_atr_activesite', 'fa_rep_activesite',
#                          'hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb',
#                          'nres', 'nres_activesite']].copy()

# for c in logit_features.columns:
#     if c in ['degree_activesite', 'nres', 'nres_activesite']:
#         continue
#     if 'catres' in c:
#         continue
#     norm_by = 'nres_activesite' if 'activesite' in c else 'nres'  
#     print(c, norm_by)
#     logit_features[c] = logit_features[c] / logit_features[norm_by]  

# logit_features['hbond'] = logit_features[['hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb']].sum(axis=1)    
# logit_features['vdw_activesite'] = logit_features.apply(lambda r: r['fa_atr_activesite'] + 0.55*r['fa_rep_activesite'], axis=1)

# logit_features = logit_features[[c for c in logit_xylanase_model.params.keys() if c != 'intercept']]

## Step 2: load ml model and use it to predict the labels

I'm using the whole-structure model I have trained using the data from the previous round

In [7]:
import statsmodels.api as smapi
logit_xylanase_model = smapi.load('../ml_models/logit_xylanases_structure.pickle')

In [8]:
scaler = preprocessing.StandardScaler()

X = pd.DataFrame(scaler.fit_transform(logit_features[['total_score', 'hbond', 'p_aa_pp', 'yhh_planarity', 'contacts']]), 
                 columns=['total_score', 'hbond', 'p_aa_pp', 'yhh_planarity', 'contacts'])

X['intercept'] = 1.0  # so we don't need to use sm.add_constant every time
X.set_index(logit_features.index, inplace=True)

In [9]:
def predict(modelParams, X, threshold=0.88):  
    y_proba = modelParams.predict(X[modelParams.params.index.tolist()], transform=False)
    data = pd.DataFrame(y_proba, columns=['proba'])
    data['pred'] = data.proba.apply(lambda x: 1 if x >= threshold else 0)
    return data

In [10]:
predictions = predict(logit_xylanase_model, X[logit_xylanase_model.params.index.tolist()])
predictions.pred.sum()

4178

In [11]:
logit_features = logit_features[['total_score', 'hbond', 'p_aa_pp', 'yhh_planarity', 'contacts']]
logit_features = pd.merge(logit_features, predictions, right_index=True, left_index=True)